# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. Using DeepLabCut for 3D markerless pose estimation during behavior across species.

pre-print: https://www.biorxiv.org/content/10.1101/476531v1

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut
import os

In [2]:
os.getcwd()

'/home/braincreator/Pose_estimation/DeepLabCut/examples'

In [3]:
task='Testing' # Enter the name of your experiment Task
experimenter='Shan' # Enter the name of the experimenter
video=['videos/ostrich.mp4'] # Enter the paths of your videos you want to grab frames from.

path_config_file=deeplabcut.create_new_project(task, experimenter, video, working_directory= os.getcwd(),copy_videos=True) 
#change the working directory to where you want the project created.
if path_config_file == None:
#     path_config_file = '/home/braincreator/Pose_estimation/DeepLabCut/examples/Testing-Shan-2019-07-30/config.yaml'
    path_config_file = '/home/braincreator/Pose_estimation/DeepLabCut/examples/ostrich/Testing-Shan-2019-08-07/config.yaml'
print(path_config_file)
# The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
#path_config_file = '/home/Mackenzie/Reaching/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

Project "/home/braincreator/Pose_estimation/DeepLabCut/examples/Testing-Shan-2019-08-08" already exists!
/home/braincreator/Pose_estimation/DeepLabCut/examples/ostrich/Testing-Shan-2019-08-07/config.yaml


#### Note that you can see more information about ANY function by adding a ? at the end,  i.e. 

In [4]:
deeplabcut.extract_frames?

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [4]:
%matplotlib inline
#there are other ways to grab frames, such as uniformly; please see the paper:

#AUTOMATIC:
# deeplabcut.extract_frames(path_config_file, 'automatic', crop = True) 

#AND/OR:
#SELECT RARE EVENTS MANUALLY:
%gui wx
deeplabcut.extract_frames(path_config_file,'manual')

Quitting for now!

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


## Label the extracted frames

Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [4]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Shan.
/home/braincreator/Pose_estimation/DeepLabCut/examples/ostrich/Testing-Shan-2019-08-07/labeled-data/ostrich_labeled  already exists!
They are stored in the following folder: /home/braincreator/Pose_estimation/DeepLabCut/examples/ostrich/Testing-Shan-2019-08-07/labeled-data/ostrich_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use relauch the labeling GUI to move them around, save, and re-plot!

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [5]:
deeplabcut.create_training_dataset(path_config_file)

/home/braincreator/Pose_estimation/DeepLabCut/examples/Testing-Shan-2019-08-01/training-datasets/iteration-0/UnaugmentedDataSet_TestingAug1  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [6]:
deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=100, maxiters=3000)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7]],
 'all_joints_names': ['nose',
                      'head',
                      'neck',
                      'tail',
                      'backright',
                      'backleft',
                      'frontright',
                      'frontleft'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_TestingAug1/Testing_Shan90shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/braincreator/anaconda3/envs/dlc/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': True,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2

Starting with standard pose-dataset loader.


KeyError: 'resnet_v1_50/block3/unit_12/bottleneck_v1'

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [7]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_TestingJul29/Testing_Shan95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 200,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/braincreator/anaconda3/envs/dlc/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_TestingJul29/Documentation_data-Testing_95shuffle

Running  DeepCut_resnet50_TestingJul29shuffle1_600  with # of trainingiterations: 600


0it [00:00, ?it/s]

Analyzing data...


20it [00:09,  2.16it/s]


Done and results stored for snapshot:  snapshot-600
Results for 600  training iterations: 95 1 train error: 6.91 pixels. Test error: 8.13  pixels.
With pcutoff of 0.1  train error: 6.91 pixels. Test error: 8.13 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [1]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip('videos/cow_video.mp4', 0, 120, targetname="videos/cut_portion.mp4")


[MoviePy] Running:
>>> /home/braincreator/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1 -y -i videos/cow_video.mp4 -ss 0.00 -t 120.00 -vcodec copy -acodec copy videos/cut_portion.mp4
... command successful.


In [2]:
videofile_path = ['videos/cut_portion.mp4'] #Enter a folder OR a list of videos to analyze.
path_config_file = '/home/braincreator/Pose_estimation/DeepLabCut/examples/Testing-Shan-2019-07-29/config.yaml'
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype= '.mp4')

Using snapshot-600 for model /home/braincreator/Pose_estimation/DeepLabCut/examples/Testing-Shan-2019-07-29/dlc-models/iteration-0/TestingJul29-trainset95shuffle1


W0730 12:40:29.330207 139895472727872 deprecation.py:323] From /home/braincreator/anaconda3/envs/dlc/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Starting to analyze %  videos/cut_portion.mp4
Video already analyzed! videos/cut_portionDeepCut_resnet50_TestingJul29shuffle1_600.h5
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DeepCut_resnet50_TestingJul29shuffle1_600'

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [3]:
deeplabcut.extract_outlier_frames?

In [0]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [0]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [0]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [0]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [3]:
deeplabcut.create_labeled_video(path_config_file, videofile_path[0],outputframerate = 5)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [20]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

UsageError: unrecognized arguments: #for making interactive plots.


In [5]:
!pip install pprint

  Stored in directory: /home/braincreator/.cache/pip/wheels/42/d4/c6/16a6495aecc1bda5d5857bd036efd50617789ba9bea4a05124
Successfully built pprint
